In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from statistics import mean 
import nltk

In [2]:
train = pd.read_csv("data/train_data.csv")
dev = pd.read_csv("data/dev_data.csv")
train["tweet"] = train["tweet"].astype(str)
dev["tweet"] = dev["tweet"].astype(str)

In [3]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def lemmatize_sentence(tweet):
    word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
    tokenized_sentence = word_tokenizer.tokenize(tweet)
    tokens = [lemmatize(token) for token in tokenized_sentence]
    return " ".join(tokens)
    
def remove_punct(tweet):
    punctuation = r"""[.\/#!$%\^&\*;\=\?\]\[:{}=\-_`~()\'\！\’"|]"""
    tweet = re.sub(punctuation, "", tweet)
    return tweet

def preprocess(tweet):
    tweet = remove_punct(tweet)
    #tweet = lemmatize_sentence(tweet)
    return tweet

In [4]:
#train["tweet"].apply(preprocess)
#dev["tweet"].apply(preprocess)

In [5]:
train.sort_values(by = "author_Id")
concatenated_train = train.groupby('author_Id').agg({'tweet': '\n'.join, 
                                                     'num_words':mean,
                                                     'ratio_stopwords':mean,
                                                     'ratio_upper':mean,
                                                     'average_word_length':mean,
                                                     'is_retweet': sum,
                                                     'is_modified_retweet': sum,
                                                     'ratio_emoticon':mean,
                                                     'ratio_punctuations':mean,
                                                     })
# ['tweet'].apply(' '.join).reset_index()


In [6]:
concatenated_train = concatenated_train.sample(frac=1).reset_index(drop=False)
concatenated_train[:1]

author_Id                                              tweet  num_words  \
0       7412  #classicmoviequotes "I want my two dollars...t...  13.414634   

   ratio_stopwords  ratio_upper  average_word_length  is_retweet  \
0         0.163518     0.066187             6.660616         4.0   

   is_modified_retweet  ratio_emoticon  ratio_punctuations  
0                  0.0        0.009103            0.076152

In [7]:
train_X = concatenated_train.loc[:, concatenated_train.columns != 'author_Id']
train_y = concatenated_train['author_Id']

In [8]:
vectorizer = TfidfVectorizer( 
                                decode_error=u'strict', strip_accents=None, 
                                preprocessor=None, 
                                tokenizer=None, analyzer=u'char', 
                                stop_words=None, 
                                ngram_range=(2, 3), max_df=1.0, 
                                min_df=1, max_features=8000, 
                                vocabulary=None, binary=False,  
                                norm=u'l2', use_idf=True, 
                                smooth_idf=True, sublinear_tf=False)

In [9]:
column_trans = ColumnTransformer([('text', vectorizer,'tweet')],
                                 remainder='drop')
     
    

In [10]:
dev_X = dev.loc[:, concatenated_train.columns != 'author_Id']

In [11]:
column_trans_train = column_trans.fit_transform(train_X, train_y)
#column_trans_dev = column_trans.fit_transform(dev_X)

In [12]:
column_trans_dev_X = column_trans.transform(dev_X)

In [13]:
clf = LinearSVC()
clf.fit(column_trans_train, train_y)
#clfprob = clf.predict_proba(dev_X)
predict_y = clf.predict(column_trans_dev_X)


In [14]:
#print out he evaluate values
dev_y = dev['author_Id']
def classifier_report(true_y, predict_y):
    
    print("The accuracy: ",accuracy_score(true_y, predict_y))
    print("The precision: ", precision_score(true_y, predict_y, average = "macro"))
    print("The recall: ", recall_score(true_y, predict_y, average = "macro"))
    print("The f1-score: ", f1_score(true_y, predict_y, average = "macro"))
    
    return 
classifier_report(dev_y,predict_y)

The accuracy:  0.1771551566889205


/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The precision:  0.19117497908807138
The recall:  0.16500432640710527
The f1-score:  0.12811113379960493


/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
import pickle

In [16]:
filename = 'concatedFeature_svm_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [21]:
def data_preprocess(data_line):
    hashtag=r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)" # hash-tags
    #data_line = re.sub(hashtag, "", data_line)
    data_line = re.sub(r'http\S+', 'http', data_line)
    #data_line = data_line.replace("@handle","handle")
    data_line = data_line.replace("\n","")
    return data_line

In [22]:
def read_prep(tweets_file):

    tweet = [] 


    with open(tweets_file, "r") as file:
        for i in file:
            data_line = i
            tweet.append( data_preprocess(data_line))

    df = pd.DataFrame({
                       'tweet':tweet})
    return df

In [25]:
predict_file = "data/test_tweets_unlabeled.txt"
predict_text = read_prep(predict_file)
#pred_text = predict_text["tweet"].astype(str)
predict_X = column_trans.transform(predict_text)
predict_y = clf.predict(predict_X)

In [26]:
print(predict_y)

[ 691  544 6492 ... 4181 2817 1268]


In [27]:
submit_file=open("submit2.txt", "a+")

index = 0
Predicted = []
Id = list(range(1,35438))
with open(predict_file, "r") as file:
    for line in file:
        Predicted.append(line)

df = pd.DataFrame({'Id':Id,'Predicted':predict_y})
#df["Id"] = df["Id"].astype(str)
df.to_csv("submit.csv",index = False)
        
submit_file.close()